In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dem_stitcher.stitcher import stitch_dem
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
from dem_stitcher.rio_tools import reproject_arr_to_match_profile
import numpy as np
import pandas as pd
import numpy as np

In [3]:
# Odessa Texas
bounds = [-103, 31, -101.5, 33] 
dem_0 = 'glo_30'
dem_1 = 'nasadem'
ellipsoidal_hgt = True

# Loading the 1st DEM

In [ ]:
%%time

X_0, p_0 = stitch_dem(bounds, 
                      dem_0, 
                      # More than 5 workers can cause timeout errors.
                      n_threads_downloading=5,
                      dst_ellipsoidal_height=ellipsoidal_hgt)
p_0

Reading glo_30 Datasets: 100%|██| 12/12 [00:16<00:00,  1.37s/it]


In [ ]:
# source https://stackoverflow.com/a/70334691
fig, ax = plt.subplots()
data = plot.show(X_0, transform=p_0['transform'], ax=ax)
im = data.get_images()[0]
fig.colorbar(im, label='Ellipsoidal Height (m)', ax=ax)

# Loading the 2nd DEM

In [ ]:
%%time

X_1, p_1 = stitch_dem(bounds, 
                      dem_1, 
                      # More than 5 workers can cause timeout errors.
                      max_workers=5,
                      dst_ellipsoidal_height=ellipsoidal_hgt)
p_1

# Comparison

We first have to reproject the two DEMs into the same frame; although they were called using the same bounds, dem-stitcher aligns to |the original pixel resolution.

In [ ]:
X_1_r, _ = reproject_arr_to_match_profile(X_1, p_1, p_0, resampling='bilinear')
X_1_r = X_1_r[0, ...]

We get the qualitative statistics of the difference of the two DEMs.

In [ ]:
diff = (X_0 - X_1_r)
diff_r = diff[~np.isnan(diff)]
plt.hist(diff_r, bins=50, range=(-10, 10))
plt.xlabel(f'{dem_1} - {dem_0} (m)')

In [ ]:
df = pd.DataFrame({'diff': diff_r})
df['diff_cm'] = np.round(df['diff'] * 100, 3)
df.drop(columns=['diff'], inplace=True)
df.describe()

In [ ]:
fig, ax = plt.subplots()
data = plot.show(diff, transform=p_0['transform'], ax=ax, vmin=-10, vmax=10)
im = data.get_images()[0]
fig.colorbar(im, label='Ellipsoidal Height (m)', ax=ax)